# RSS default params

self.alpha_lon_accel_max = 3.5 

self.alpha_lon_brake_max = 8.0

self.alpha_lon_brake_min = 4.0

self.alpha_lon_brake_min_correct = 3.0

self.alpha_lat_accel_max = 0.2

self.alpha_lat_brake_min = 0.8

self.lateral_fluctuation_margin = 0.0

self.response_time = 1.0

In [1]:
# from sklearn.cluster import KMeans 
import pandas as pd
import os
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d.axes3d import Axes3D

import numpy as np
from matplotlib import cm

import csv

import sklearn.cluster
from sklearn.mixture import GaussianMixture
from itertools import combinations
import math

from sklearn.neighbors import KernelDensity

# for interactive plots
%matplotlib qt

# Run Kmeans and get centroids

In [2]:
#### Names
scenario_names = ['LVM1', 'LVM2', 'LVDAD', 'LVD', 'POV_UnprotectedLeft', 'OppositeVehicleRunningRedLight', 'all']

X1_name = 'alpha_lon_accel_max'
X2_name = 'alpha_lon_brake_max'
X3_name = 'alpha_lon_brake_min'
X4_name = 'response_time'
X5_name = 'alpha_lat_accel_max'
X6_name = 'alpha_lat_brake_min'
X7_name = 'lateral_fluctuation_margin'
X8_name = 'alpha_lon_brake_min_correct'
rob = 'robustness'

var_names = [rob, X1_name, X2_name, X3_name, X4_name, X5_name, X6_name, X7_name, X8_name]
var_units = ['m/s^2','m/s^2', 'm/s^2', 's', 'm/s^2', 'm/s^2', 'm', 'm/s^2' , 'm']
## concat all data
data_concat = pd.DataFrame()
data_all = []

for sce in scenario_names[:-1]:
    scenario = sce
    res_path = "../../results/"+scenario
    data_aux = pd.read_csv(os.path.join(res_path, scenario + '_combined.csv'), index_col=False)
    
    data_all.append(data_aux)
    data_concat = pd.concat([data_concat, data_aux], ignore_index=True, sort=False)

data_concat.to_csv("../../results/all_combined.csv", index=False)

data_collision = data_concat.loc[data_concat[rob] < 0.1]
data_no_collision = data_concat.loc[data_concat[rob] >= 0.1]

data_all.append(data_concat)
print(len(data_all))

7


In [ ]:
data_no_collision.cov()

In [ ]:
data_collision.cov()

#### Kernel Density Estimation
https://scikit-learn.org/stable/auto_examples/neighbors/plot_kde_1d.html#sphx-glr-auto-examples-neighbors-plot-kde-1d-py

In [ ]:
samples = data_collision
var_names = [X1_name, X2_name, X3_name, X4_name, X5_name, X6_name, X7_name] #, X8_name]

for i, var in enumerate(var_names[]):
    print(var)
    X = samples[var].dropna()
    X = np.array(X)
    if X.shape[0] > 0:
        print(X.shape)

        X = X.reshape(-1, 1)
        N = X.shape[0]

        kde = KernelDensity(kernel='gaussian', bandwidth=0.5).fit(X)

        max_point = X.max()
        
        X_plot = np.linspace(0, max_point, 3000)[:, None]

        log_dens = kde.score_samples(X_plot)
        
        fig = plt.figure(i, figsize=(7,4))
        ax = fig.subplots()
        ax.plot(X_plot[:, 0], np.exp(log_dens), color = 'black', linestyle='-', label="KDE {0}, {1} pts".format(var, N))

        ax.legend(loc='upper right')        
        ax.set_xlabel(var + ' [' + var_units[i] + ']', fontsize=10)

        figname = os.path.join("../../results", 'all_collision_kde_' +  var + '.png')
        plt.savefig(figname, dpi=200)
        
        plt.show()
    else: 
        print(var, 'no collision')

#### Run kmeans and get clusters

In [ ]:
check_collision = 0
lat_flag = 0
all_flag = 0

if check_collision:
    fn_centroids_all_scenarios = os.path.join("../../results/", 'collision_global_centroids.csv')
else:
    fn_centroids_all_scenarios = os.path.join("../../results/", 'global_centroids.csv')
      
# fh_all_centroids = open(fn_centroids_all_scenarios, 'w')
# writer_all_centroids = csv.writer(fh_all_centroids)
# writer_all_centroids.writerow([rob, X1_name, X2_name, X3_name, X4_name, X5_name, X6_name, X7_name, X8_name])

# centroids_df = pd.DataFrame(columns=[rob, X1_name, X2_name, X3_name, X4_name, X5_name, X6_name, X7_name, X8_name])
centroids_df = pd.DataFrame()

centroids_all = []

for i, data_ in enumerate(data_all[:-1]):
    
    scenario = scenario_names[i]
    
    if scenario == 'all':
        res_path = "../../results/"
    else:
        res_path = "../../results/"+scenario
    
    if scenario == 'OppositeVehicleRunningRedLight':
        lat_flag = 1
    elif scenario == 'POV_UnprotectedLeft':
        all_flag = 0
    else:
        lat_flag = 0

    if lat_flag:
        var_names = [rob, X1_name, X2_name, X3_name, X4_name, X5_name, X6_name, X7_name]
    elif all_flag:
        var_names = [rob, X1_name, X2_name, X3_name, X4_name, X5_name, X6_name, X7_name, X8_name]
    else:
        var_names = [rob, X1_name, X2_name, X3_name, X4_name]

    print(scenario)
    scenario_fn = scenario

    data_ = data_.sort_values(rob)
    if check_collision:
        data = data_.loc[data_[rob] < 0.1]
        scenario_fn = scenario + '_collision'

        if scenario == 'all':
            file_name = os.path.join(res_path, scenario_fn + '_combined.csv')
            data.to_csv(file_name, index=False)
        
        clusters = 1
    else:
        data = data_.loc[data_[rob] >= 0.1]
        clusters = 4
    
    data_desc = data.describe()
    file_name = os.path.join(res_path, scenario_fn + '_desc.csv')
    data_desc.to_csv(file_name, index=True)

    if data.shape[0] > 0:
        #### Run Kmeans
        
        model = sklearn.cluster.KMeans(n_clusters = clusters, n_jobs = 4)
        labels = model.fit_predict(data)
        centroids = model.cluster_centers_

        ## Save centorids
        file_name = os.path.join(res_path, scenario_fn + '_centroid.csv')

        print("saving centroids to", file_name)

        centroids_aux = centroids.tolist()
        
        ## concat all centroids
        df = pd.DataFrame(centroids_aux, columns = var_names)
        df['scenario'] = [scenario]*clusters
        centroids_df = pd.concat([centroids_df, df], ignore_index=True, sort=False)

        print(centroids_df)
        
        res = [var_names] + centroids_aux
        with open(file_name, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerows(res)

        ## save data with cluste
        file_name = os.path.join(res_path, scenario_fn + '_param_cluster.csv')

        print('Saving labels to', file_name)

        data['cluster_label'] = labels  
        data.to_csv(file_name, index=False)

        #### Run combination of var
        comb_names = combinations([X1_name, X2_name, X3_name, X4_name], 2) 
        names_list = list(comb_names)

        rob = 'robustness'
        R = data[rob]

        ## Plot 3D graphs
        for i, var in enumerate(names_list):
            search_name_x = names_list[i][0]
            search_name_y = names_list[i][1]

            X1 = data[search_name_x]
            X2 = data[search_name_y]

            ## Plot data
            fig = plt.figure(i)
            ax = plt.axes(projection='3d')    

            ax.scatter3D(X1, X2, R, c=labels, cmap=cm.RdYlGn, alpha=0.8)
            ax.set_xlabel(search_name_x, fontsize=15)
            ax.set_ylabel(search_name_y, fontsize=15)
            ax.set_zlabel(rob, fontsize=15)
            plt.tick_params(labelsize=10) 

            figname = os.path.join(res_path, scenario_fn + '_kmeans_' +  search_name_x + '_vs_' + search_name_y + '.png')

            plt.savefig(figname, dpi=200)

# plt.show()

print("saving centroids to", fn_centroids_all_scenarios)
centroids_df.to_csv(fn_centroids_all_scenarios, index=True)
# fh_all_centroids.close()

LVM1
saving centroids to ../../results/LVM1/LVM1_centroid.csv
   robustness  alpha_lon_accel_max  alpha_lon_brake_max  alpha_lon_brake_min  \
0   20.136153             3.972561             9.638163             3.976664   
1   84.417363             7.010354             9.781903             3.748716   
2   46.895913             5.489969             9.875495             4.006912   
3    3.625738             1.675868             7.319542             4.071455   

   response_time scenario  
0       2.369595     LVM1  
1       3.590225     LVM1  
2       2.669986     LVM1  
3       0.441394     LVM1  
Saving labels to ../../results/LVM1/LVM1_param_cluster.csv


/Users/melli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


LVM2
saving centroids to ../../results/LVM2/LVM2_centroid.csv
   robustness  alpha_lon_accel_max  alpha_lon_brake_max  alpha_lon_brake_min  \
0   20.136153             3.972561             9.638163             3.976664   
1   84.417363             7.010354             9.781903             3.748716   
2   46.895913             5.489969             9.875495             4.006912   
3    3.625738             1.675868             7.319542             4.071455   
4    3.082534             2.833313             7.892469             5.408606   
5   79.541344             7.328585             9.826596             3.928073   
6   18.232121             4.639829             9.417247             4.574767   
7   43.606844             5.434260             9.794116             4.464149   

   response_time scenario  
0       2.369595     LVM1  
1       3.590225     LVM1  
2       2.669986     LVM1  
3       0.441394     LVM1  
4       0.462789     LVM2  
5       3.532728     LVM2  
6       2.214687     

/Users/melli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


LVDAD
saving centroids to ../../results/LVDAD/LVDAD_centroid.csv
    robustness  alpha_lon_accel_max  alpha_lon_brake_max  alpha_lon_brake_min  \
0    20.136153             3.972561             9.638163             3.976664   
1    84.417363             7.010354             9.781903             3.748716   
2    46.895913             5.489969             9.875495             4.006912   
3     3.625738             1.675868             7.319542             4.071455   
4     3.082534             2.833313             7.892469             5.408606   
5    79.541344             7.328585             9.826596             3.928073   
6    18.232121             4.639829             9.417247             4.574767   
7    43.606844             5.434260             9.794116             4.464149   
8     3.301890             7.412755             7.991522             5.515613   
9    19.494198             6.212037             9.669511             4.277885   
10    2.732188             1.742116         

/Users/melli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


LVD
saving centroids to ../../results/LVD/LVD_centroid.csv
    robustness  alpha_lon_accel_max  alpha_lon_brake_max  alpha_lon_brake_min  \
0    20.136153             3.972561             9.638163             3.976664   
1    84.417363             7.010354             9.781903             3.748716   
2    46.895913             5.489969             9.875495             4.006912   
3     3.625738             1.675868             7.319542             4.071455   
4     3.082534             2.833313             7.892469             5.408606   
5    79.541344             7.328585             9.826596             3.928073   
6    18.232121             4.639829             9.417247             4.574767   
7    43.606844             5.434260             9.794116             4.464149   
8     3.301890             7.412755             7.991522             5.515613   
9    19.494198             6.212037             9.669511             4.277885   
10    2.732188             1.742116             7.

In [ ]:
print("saving centroids to", fn_centroids_all_scenarios)
centroids_df.to_csv(fn_centroids_all_scenarios, index=True)

print(centroids_df)

In [ ]:
df = pd.read_excel('hidro_ne.xlsx')

fig, ax = plt.subplots()
ax.plot(list(range(1,13)), df[2015].values, 'r', linewidth=2)
ax.legend(['2015'])
df.T.plot.box(yticks=range(0, 105, 5), ax=ax)

In [ ]:
centroids_aux[0][1:]

In [ ]:
print(var_names)

plt.plot(var_names[1:], centroids_aux[0][1:], '+k-', label="centroids")# {0}, {1} pts".format(var, N))

data.boxplot(rot=45, fontsize=10, figsize=(15, 20), column = var_names[1:])


# Experimental

In [ ]:
scenario_names = ['LVM1', 'LVM2', 'LVDAD', 'LVD', 'POV_UnprotectedLeft']
scenario = scenario_names[1]
# print(scenario)
#
res_path = "../../results/"+scenario

X1_name = 'alpha_lon_accel_max'
X2_name = 'alpha_lon_brake_max'
X3_name = 'alpha_lon_brake_min'
X4_name = 'response_time'

# Read data from a csv to pandas data frame
data = pd.read_csv(os.path.join(res_path, scenario + '_combined.csv'), index_col=False)

In [ ]:
from matplotlib.text import Annotation
from mpl_toolkits.mplot3d.proj3d import proj_transform

class Annotation3D(Annotation):
    def __init__(self, text, xyz, *args, **kwargs):
        super().__init__(text, xy=(0,0), *args, **kwargs)
        self._xyz = xyz

    def draw(self, renderer):
        x2, y2, z2 = proj_transform(*self._xyz, renderer.M)
        self.xy=(x2,y2)
        super().draw(renderer)


def _annotate3D(ax,text, xyz, *args, **kwargs):
    '''Add anotation `text` to an `Axes3d` instance.'''

    annotation= Annotation3D(text, xyz, *args, **kwargs)
    ax.add_artist(annotation)

setattr(Axes3D,'annotate3D',_annotate3D)

In [ ]:
search_name_x = "alpha_lon_brake_min"
search_name_y = 'alpha_lon_brake_max'
rob = 'robustness'
X1 = data[search_name_x]
X2 = data[search_name_y]
R = data[rob]

In [ ]:
## concatenate data from all scenarios
concat_flag = 1

data_all = pd.DataFrame()

data = []
for sce in scenario_names:
    print(sce)
    scenario = sce
    res_path = "../../results/"+scenario
    data_aux = pd.read_csv(os.path.join(res_path, scenario + '_combined.csv'), index_col=False)
    data.append(data_aux)
    
    if concat_flag:
        print("concatenation of all scenarios")
        data_all = pd.concat([data_all, data], ignore_index=True)
        scenario = "all"
        res_path = "../../results/"
        
        data.append(data_all)

# Run Kmeans

In [ ]:

clusters = 3

kmeans = KMeans(n_clusters = clusters, n_jobs=4) 
kmeans.fit(data) 

km_labels = kmeans.labels_
km_centers = kmeans.cluster_centers_

# Create the pandas DataFrame  
km_centers_df = pd.DataFrame(km_centers, columns = [X1_name, X2_name, X3_name, X4_name, 'robustness'])
print(df)
print((km_labels))

data['cluster_label'] = km_labels
data = data.sort_values(by=['cluster_label'])

data.to_csv(os.path.join(res_path, scenario + '_param_cluster.csv'), index=False)

In [ ]:
### Plot kmeans centroids

ax = plt.axes(projection='3d')    
fontsize =15
ax.scatter3D(X1, X2, R, c=R, cmap=cm.RdYlGn, alpha=0.3)
ax.set_xlabel(search_name_x, fontsize=fontsize)
ax.set_ylabel(search_name_y, fontsize=fontsize)
ax.set_zlabel('Robustness', fontsize=fontsize)
plt.tick_params(labelsize=10) 

ax.scatter3D(km_centers_df[search_name_x], km_centers_df[search_name_y], km_centers_df[rob], marker='o', s=1000, c='magenta')

for i, txt in enumerate(km_centers_df[rob]):
    print(txt)
    
#     ax.text(km_centers_df[search_name_x].iloc[i], km_centers_df[search_name_y].iloc[i]
#             , km_centers_df[rob].iloc[i], str(km_centers_df[rob].iloc[i]), color='black')
    
    ax.annotate3D(str(km_centers_df[rob].iloc[i])[:5],
                  (km_centers_df[search_name_x].iloc[i], km_centers_df[search_name_y].iloc[i], km_centers_df[rob].iloc[i]),
                  xytext=(-100,60),
                  textcoords='offset points',
                  bbox=dict(boxstyle="round", fc="lightyellow"),
                  arrowprops = dict(arrowstyle="-|>",ec='black', fc='white', lw=2))
    
    
plt.show()

# Experimental

In [ ]:
from sklearn.decomposition import PCA 

pca = PCA(3) 
model = pca.fit(data) 

pca_data = pd.DataFrame(pca.transform(data)) 

print(model.singular_values_)


from matplotlib import colors as mcolors 
import math 

''' Generating different colors in ascending order of their hsv values '''
colors = list(zip(*sorted((tuple(mcolors.rgb_to_hsv(
    mcolors.to_rgba(color)[:3])), name)
                           for name, color in dict( 
                               mcolors.BASE_COLORS, **mcolors.CSS4_COLORS
                           ).items())))[1] 


# number of steps to taken generate n(clusters) colors 
skips = math.floor(len(colors[5 : -5])/clusters) 
cluster_colors = colors[5 : -5 : skips] 



In [ ]:
from mpl_toolkits.mplot3d import Axes3D 
import matplotlib.pyplot as plt 
%matplotlib qt

fig = plt.figure() 
ax = fig.add_subplot(111, projection = '3d') 
ax.scatter(pca_data[1], pca_data[2], pca_data[0], 
           c = list(map(lambda label : cluster_colors[label], 
                        kmeans.labels_))) 

str_labels = list(map(lambda label:'% s' % label, kmeans.labels_)) 

# list(map(lambda data1, data2, data3, str_label:
#          ax.text(data1, data2, data3, s = str_label, size = 20,
#                  zorder = 20, color = 'k'), pca_data[0], pca_data[1],
#          pca_data[2], str_labels)) 

plt.show() 
